In [1]:
# ============================================================================
# CELLA 1: Setup Progetto Jupyter
# ============================================================================

import sys
import os
from pathlib import Path
import torch

print(" AML Semantic Correspondence - Training-Free Baseline\n")
PROJECT_ROOT = str(Path.home() / "AML")
LOCAL_REPO_NAME = str(Path.home() / "AML_SemanticCorrespondence")
DATA_DIR = f'{PROJECT_ROOT}/dataset' 
CHECKPOINT_DIR = f'{PROJECT_ROOT}/checkpoints'
RESULTS_DIR = f'{PROJECT_ROOT}/results'

# Ensure these directories exist (they will be created inside MyDrive/AML)
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

# Aggiungi al path la directory locale del repository
sys.path.insert(0, LOCAL_REPO_NAME)

# 4. Verifica GPU
print(f"\n  GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")
if torch.cuda.is_available():
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print("\n Setup complete!\n")

 AML Semantic Correspondence - Training-Free Baseline


  GPU: Tesla V100-SXM2-16GB
   VRAM: 16.9 GB

 Setup complete!



In [ ]:
# ============================================================================
# CELLA 1b: Scelta backbones, finetuning, dataset e metodo di prediction
# ============================================================================
backbone_choice = 'dinov2'  # 'dinov2', 'dinov3', 'sam'
finetune_choice = False     # True, False
soft_argmax_choice = False  # True, False
dataset_choice = 'spair'   # 'spair', 'pfwillow'
# ============================================================================

In [3]:
# ============================================================================
# CELLA 2: Install Dependencies
# ============================================================================

print(" Installing dependencies...\n")

# Installa da requirements.txt clonato
!pip install -q -r {LOCAL_REPO_NAME}/requirements.txt

print(" Dependencies installed!\n")

 Installing dependencies...

 Dependencies installed!



In [4]:
# ============================================================================
# CELLA 3: Load  Dataset
# ============================================================================

from dataset import SPairDataset, PFWillowDataset
from torch.utils.data import DataLoader
from pathlib import Path
DATASET_ROOT = f'{DATA_DIR}/Spair-71k' if dataset_choice == 'spair' else f'{DATA_DIR}/PF-WILLOW'

if not Path(DATASET_ROOT).exists():
    raise FileNotFoundError(f"Dataset not found: {DATASET_ROOT}")

if dataset_choice == 'spair':
    # Load dataset
    test_dataset = SPairDataset(
        root=DATASET_ROOT,
        split='test',
        size='large',
        long_side=518,
        normalize=True,
        load_segmentation=False
    )

    # DataLoader
    test_loader = DataLoader(
        test_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=2,
        pin_memory=torch.cuda.is_available()
    )
else:   #Preparato per poterci aggiungere PF-Willow
    # Load dataset
    test_dataset = PFWillowDataset(
        root=DATASET_ROOT,
        long_side=518,
        normalize=True
    )

    # Create DataLoader (batch_size=1 importante!)
    test_loader = DataLoader(
        test_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=2,
        collate_fn=PFWillowDataset.collate_fn
    )
# ============================================================================

Extracting category and subset from image paths...
Categories: ['car', 'duck', 'motorbike', 'winebottle']
Subsets: ['car(G)', 'car(M)', 'car(S)', 'duck(S)', 'motorbike(G)', 'motorbike(M)', 'motorbike(S)', 'winebottle(M)', 'winebottle(wC)', 'winebottle(woC)']

Loaded 900 pairs
  4 categories: ['car', 'duck', 'motorbike', 'winebottle']
  10 subsets


In [5]:
# ============================================================================
# CELLA 4: Display Available Backbones
# ============================================================================

from models import BACKBONE_REGISTRY

print("✓ Available Backbones:")
print("="*60)
for name, config in BACKBONE_REGISTRY.items():
    print(f"  • {name:15s} → {config.name} (patch={config.patch_size})")
print("="*60)
print()


✓ Available Backbones:
  • dinov2          → DINOv2-ViT-B/14 (patch=14)
  • dinov3          → DINOv3-ViT-B/16 (patch=16)
  • sam             → SAM-ViT-B (patch=16)



In [ ]:
# ============================================================================
# CELLA 5: Imports for Evaluation
# ============================================================================

from models import UnifiedBackbone, CorrespondenceMatcher, UnifiedEvaluator
import json
from pathlib import Path
from utils import compute_pck

print("✓ All modules imported successfully")


✓ All modules imported successfully


In [8]:
# ============================================================================
# CELLA 7: Main Evaluation - Run Based on Configuration
# ============================================================================

print(f"\n{'='*70}")
print("EVALUATION CONFIGURATION (from Cell 1b)")
print('='*70)
print(f"  Backbone: {backbone_choice}")
print(f"  Finetune: {finetune_choice}")
print(f"  Soft Argmax: {soft_argmax_choice}")
print(f"  Dataset: {dataset_choice}")
print('='*70)

# Initialize backbone
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\nDevice: {device}\n")

backbone = UnifiedBackbone(
    backbone_choice=backbone_choice,
    finetune_choice=finetune_choice,
    checkpoint_dir=CHECKPOINT_DIR,
    device=device
)

# Initialize matcher
matcher = CorrespondenceMatcher(
    backbone=backbone,
    use_soft_argmax=soft_argmax_choice
)

# Initialize evaluator
evaluator = UnifiedEvaluator(
    dataloader=test_loader,
    device=device,
    thresholds=[0.05, 0.10, 0.15, 0.20]
)

print("\n✓ Evaluation setup complete!")



EVALUATION CONFIGURATION (from Cell 1b)
  Backbone: dinov2
  Finetune: False
  Soft Argmax: False
  Dataset: pfwillow

Device: cuda

→ Loading DINOv2-ViT-B/14...

Using cache found in /home/jupyter/.cache/torch/hub/facebookresearch_dinov2_main
/home/jupyter/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/jupyter/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/jupyter/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


 ✓
   • Patch size: 14
   • Embedding: 768D

✓ Evaluation setup complete!


In [ ]:
# ============================================================================
# CELLA 8A: Evaluate Backbone
# ============================================================================

# Configuration for evaluation run
NUM_SAMPLES = None  # None = evaluate all, or set to e.g. 100 for testing

print(f"Evaluating {backbone_choice} on SPair-71k test set")
print(f"Samples: {NUM_SAMPLES if NUM_SAMPLES else 'ALL'}\n")

# Run evaluation
results = evaluator.evaluate(
    matcher=matcher,
    backbone_name=backbone.config.name,
    num_samples=NUM_SAMPLES,
    show_progress=True
)

# Save results
output_filename = backbone_choice
if dataset_choice == 'spair':
    output_filename += '_spair'
else:
    output_filename += '_pfwillow'
if finetune_choice:
    output_filename += '_finetuned'
else:
    output_filename += '_baseline'
if soft_argmax_choice:
    output_filename += '_softargmax'
else:
    output_filename += '_hardargmax'
output_filename += '_results.json'
output_file = Path(RESULTS_DIR) / output_filename

with open(output_file, 'w') as f:
    json.dump(results, f, indent=2)

print(f"\n✓ Results saved: {output_file}")


Evaluating dinov2 on SPair-71k test set
Samples: ALL


EVALUATING: DINOv2-ViT-B/14


DINOv2-ViT-B/14: 100%|██████████| 900/900 [01:06<00:00, 13.59it/s, PCK@0.10=0.7984]


DINOv2-ViT-B/14 Results:
----------------------------------------------------------------------
   PCK@0.05: 0.5206 ± 0.1948 (52.06%)
   PCK@0.10: 0.7984 ± 0.1718 (79.84%)
   PCK@0.15: 0.8916 ± 0.1397 (89.16%)
   PCK@0.20: 0.9342 ± 0.1069 (93.42%)

   ⏱ Avg inference time: 62.94 ms/pair
   📊 Evaluated on 900 pairs

✓ Results saved: /home/jupyter/AML/results/dinov2_results.json


In [ ]:
# ============================================================================
# CELLA 8B: Load & Compare Results (Optional)
# ============================================================================

import matplotlib.pyplot as plt
import seaborn as sns

print("Loading evaluation results...\n")

# Load results from file
if output_file.exists():
    with open(output_file, 'r') as f:
        loaded_results = json.load(f)
    
    print(f"✓ Loaded: {output_file.name}\n")
    
    # Display summary
    print("="*70)
    print(f"SUMMARY: {loaded_results['name']}")
    print("="*70)
    print(f"Samples evaluated: {loaded_results['num_pairs']}")
    print(f"Avg inference time: {loaded_results['inference_time_ms']:.2f} ms/pair\n")
    
    print("PCK Metrics:")
    for metric, vals in loaded_results['overall_keypoint'].items():
        print(f"  {metric}: {vals['mean']*100:.2f}% ± {vals['std']*100:.2f}%")
    print("="*70)
else:
    print(f"Results file not found: {output_file}")


Loading evaluation results...

✓ Loaded: dinov2_results.json

SUMMARY: DINOv2-ViT-B/14
Samples evaluated: 900
Avg inference time: 62.94 ms/pair

PCK Metrics:
  PCK@0.05: 52.06% ± 19.48%
  PCK@0.10: 79.84% ± 17.18%
  PCK@0.15: 89.16% ± 13.97%
  PCK@0.20: 93.42% ± 10.69%
